In [8]:
import os
import json
import csv
from datetime import datetime

In [9]:
curr_py_path = os.getcwd()
path = '\\DI_CONNECT\\'
path_addon = 'DI-Connect-Wellness\\'
#path_addon = 'DI-Connect-User'
#path_addon = 'DI-Connect-Wellness'
#file_name = '2017-11-25_2018-03-05_61902345_sleepData.json'

In [10]:
path = curr_py_path + path + path_addon
raw_string = r'{}'.format(path)
# Get all file names
files = os.listdir(raw_string)

In [11]:
file_paths = []
for file in files:
  # make file name 
  file_name = path + file
  file_paths.append(file_name)

In [12]:
data = []

# loop through all files
for path in file_paths:
  # make path to raw string
  raw_string = r'{}'.format(path)

  # read file to csv
  with open(raw_string, 'r', encoding="utf8") as f_handle:
    json_dump = json.load(f_handle)
    
    # loop through all entries in file
    for entry in json_dump:
      try:
        # GET DATE
        date = entry['calendarDate']
        date = datetime.strptime(date, '%Y-%m-%d')
        date = date.strftime('%d-%b-%Y')
        
        # GET SLEEP HOURS AND MINS
        start = entry['sleepStartTimestampGMT']
        end =  entry['sleepEndTimestampGMT']
        
        try:
          start = datetime.strptime(start, '%Y-%m-%dT%H:%M:%S.0')
          end = datetime.strptime(end, '%Y-%m-%dT%H:%M:%S.0')
          duration_secs = int((end - start).total_seconds())
          # if less than 1 hour
          if (duration_secs < 3600) and (duration_secs not in [0, None]) :
            sleep_hrs = 0
            sleep_mins = int(duration_secs/60)
          else:
            sleep_hrs = int(duration_secs / 3600)
            sleep_mins = int((duration_secs - (sleep_hrs * 3600)) / 60)
        except:
          sleep_hrs, sleep_mins = None, None


        # GET SLEEP METRICS
        deep_sleep_mins = entry['deepSleepSeconds']
        if deep_sleep_mins not in [0, None]:
          deep_sleep_mins = int(deep_sleep_mins / 60)

        light_sleep_mins = entry['lightSleepSeconds']
        if light_sleep_mins not in [0, None]:
          light_sleep_mins = int(light_sleep_mins / 60)

        awake_mins = entry['awakeSleepSeconds']
        if awake_mins not in [0, None]:
          awake_mins = int(awake_mins / 60)

        unmeasurable_sleep_mins = entry['unmeasurableSeconds']
        if unmeasurable_sleep_mins not in [0, None]:
          unmeasurable_sleep_mins = int(unmeasurable_sleep_mins / 60)


        # GET REM SLEEP METRICS
        try:
          rem_sleep_mins = entry['remSleepSeconds']
        except:
          rem_sleep_mins = None
          
        if rem_sleep_mins not in [0, None]:
          rem_sleep_mins = int(rem_sleep_mins / 60)


        to_save = {'date': date,
                  'sleep_hrs':sleep_hrs,
                  'sleep_mins': sleep_mins,
                  'deep_sleep_mins': deep_sleep_mins,
                  'light_sleep_mins': light_sleep_mins,
                  'rem_sleep_mins': rem_sleep_mins,
                  'awake_mins': awake_mins,
                  'unmeasurable_sleep_mins': unmeasurable_sleep_mins}

        data.append(to_save)
      except:
        pass

In [13]:
data

[{'date': '02-Nov-2017',
  'sleep_hrs': 8,
  'sleep_mins': 55,
  'deep_sleep_mins': 144,
  'light_sleep_mins': 311,
  'rem_sleep_mins': None,
  'awake_mins': 80,
  'unmeasurable_sleep_mins': 0},
 {'date': '03-Nov-2017',
  'sleep_hrs': 5,
  'sleep_mins': 21,
  'deep_sleep_mins': 75,
  'light_sleep_mins': 224,
  'rem_sleep_mins': None,
  'awake_mins': 22,
  'unmeasurable_sleep_mins': 0},
 {'date': '05-Nov-2017',
  'sleep_hrs': 8,
  'sleep_mins': 47,
  'deep_sleep_mins': 136,
  'light_sleep_mins': 266,
  'rem_sleep_mins': None,
  'awake_mins': 125,
  'unmeasurable_sleep_mins': 0},
 {'date': '06-Nov-2017',
  'sleep_hrs': 7,
  'sleep_mins': 16,
  'deep_sleep_mins': 174,
  'light_sleep_mins': 220,
  'rem_sleep_mins': None,
  'awake_mins': 42,
  'unmeasurable_sleep_mins': 0},
 {'date': '07-Nov-2017',
  'sleep_hrs': 5,
  'sleep_mins': 2,
  'deep_sleep_mins': 78,
  'light_sleep_mins': 164,
  'rem_sleep_mins': None,
  'awake_mins': 60,
  'unmeasurable_sleep_mins': 0},
 {'date': '08-Nov-2017',
  

In [14]:
# field names in new csv
field_names = ['date', 'sleep_hrs', 'sleep_mins', 'deep_sleep_mins', 'light_sleep_mins', 'rem_sleep_mins', 'awake_mins', 'unmeasurable_sleep_mins']

# Save data to new file
with open('sleep_data.csv', 'w', encoding="utf8", newline='') as csv_file:
  writer = csv.DictWriter(csv_file, fieldnames=field_names)
  writer.writeheader()
  
  # loop through list of data
  for entry in data:
    writer.writerow({
      'date': entry['date'],
      'sleep_hrs': entry['sleep_hrs'],
      'sleep_mins': entry['sleep_mins'],
      'deep_sleep_mins': entry['deep_sleep_mins'],
      'light_sleep_mins': entry['light_sleep_mins'],
      'rem_sleep_mins': entry['rem_sleep_mins'],
      'awake_mins': entry['awake_mins'],
      'unmeasurable_sleep_mins': entry['unmeasurable_sleep_mins'],
    })